In [26]:
import os
from pprint import pprint
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import aux as aux
from dask.distributed import SSHCluster,Client
import anomalies as an
import pandas as pd

In [27]:
client = Client("10.67.22.165:42429")

In [28]:
path=os.path.abspath('/home/data/data.csv')

df = dd.read_csv(path)

In [29]:
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
metrics = ['S41','S109','S100','S101','S102','S106','S107','S108','S122','S124','S126']

In [30]:
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[0], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [31]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[3], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S41
Calculating for metric S109
Calculating for metric S100


In [ ]:
def corr_finder(engine, metric_list, window):
    
    corr_coeff_pearson = []
    corr_coeff_spearman = []
    corr_coeff_kendall = []

    for k in range(len(metric_list)):
        corr_coeff_pearson.append([])
        corr_coeff_spearman.append([])
        corr_coeff_kendall.append([])

    for i in range(len(engine['Anomaly'])):
        if engine.iloc[i,1]:
            values_engine = list(engine.iloc[i-window:i+window+1,2])
            for j,metric in enumerate(metric_list):
                values_metric = list(metric.iloc[i-window:i+window+1,1])

                df = pd.DataFrame({'values_engine': values_engine, 'values_metric': values_metric})
                corr_coeff_pearson[j].append(df.corr(method='pearson').iloc[0, 1])
                corr_coeff_spearman[j].append(df.corr(method='spearman').iloc[0, 1])
                corr_coeff_kendall[j].append(df.corr(method='kendall').iloc[0, 1])
    return corr_coeff_pearson,corr_coeff_spearman,corr_coeff_kendall

In [ ]:
pearson,spearman,kendall=corr_finder(engine_norm,metric_list,window=50)

In [ ]:
with open(f"{hardware_list[3]}_{engine_list[0]}_correlations.txt",'w') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [43]:
df_fourmotors=df[df['metric'].isin(['S117','S118','S169','S170'])]

In [44]:
aux.find_anomaly_number(df_fourmotors)

hw: SW-065 metric: S117
The instances of 1 single isolated uptime or downtime reading are: 0
The instances of uptimes or downtimes between 30 sec and 1 min are: 2
The instances of <30 seconds uptimes or downtimes are: 2
hw: SW-065 metric: S169
The instances of 1 single isolated uptime or downtime reading are: 0
The instances of uptimes or downtimes between 30 sec and 1 min are: 1
The instances of <30 seconds uptimes or downtimes are: 0
hw: SW-065 metric: S118
The instances of 1 single isolated uptime or downtime reading are: 0
The instances of uptimes or downtimes between 30 sec and 1 min are: 0
The instances of <30 seconds uptimes or downtimes are: 0
hw: SW-065 metric: S170
The instances of 1 single isolated uptime or downtime reading are: 0
The instances of uptimes or downtimes between 30 sec and 1 min are: 0
The instances of <30 seconds uptimes or downtimes are: 0
hw: SW-088 metric: S169
The instances of 1 single isolated uptime or downtime reading are: 0
The instances of uptimes or